In [153]:
import os
from resolve_api_samples.Examples.python_get_resolve import GetResolve
from pprint import pprint
import sys
from typing import List
from collections import Counter

resolve = GetResolve()
project_manager = resolve.GetProjectManager()
project = project_manager.GetCurrentProject()
media_storage = resolve.GetMediaStorage()
media_pool = project.GetMediaPool()
root_folder = media_pool.GetRootFolder()

media_path = '/Volumes/thom_7_for_Mac/tmp/素材'
sub_folders_full_path = media_storage.GetSubFolderList(media_path)
INVALID_EXTENSION = ["DS_Store", "JPG", "JPEG", "SRT"]  # TODO, 小写的情况还待考虑进去


def get_sub_folder_by_name(sub_folder_name: str):
    """Get folder object by name."""
    all_sub_folder = root_folder.GetSubFolderList()
    sub_folder_dict = {sub_folder.GetName(): sub_folder for sub_folder in all_sub_folder}
    return sub_folder_dict.get(sub_folder_name, "")


def absolute_file_paths(directory) -> list:
    absolute_file_path_list = []
    for directory_path, _, filenames in os.walk(directory):
        for f in filenames:
            absolute_file_path_list.append(os.path.abspath(os.path.join(directory_path, f)))
    return absolute_file_path_list


def import_clip() -> None:
    """Import footage from media storage into the corresponding sub-folder of the media pool root folder."""
    for cam_path in sub_folders_full_path:
        filename_and_fullpath_dict = {os.path.splitext(path)[0].replace(".", "").split('/')[-1]: path for path in
                                      absolute_file_paths(cam_path) if path.split('.')[-1] not in INVALID_EXTENSION}
        filename_and_fullpath_keys = list(filename_and_fullpath_dict.keys())
        filename_and_fullpath_keys.sort()
        filename_and_fullpath_value = [filename_and_fullpath_dict.get(i) for i in filename_and_fullpath_keys]
        current_folder = get_sub_folder_by_name(f"{cam_path.split('/')[cam_path.split('/').index('素材') + 1]}")
        media_pool.SetCurrentFolder(current_folder)
        media_storage.AddItemListToMediaPool(filename_and_fullpath_value)


import_clip()